In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [2]:
df0 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.000.csv")
df1 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.001.csv")
df2 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.002.csv")
df3 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.003.csv")
df4 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.004.csv")
df5 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.005.csv")
df6 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.006.csv")
df7 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.007.csv")
df8 = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Parking_Tags_Data_2019.008.csv")
df = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8], axis=0, ignore_index=True)

In [3]:
#Delete instances with no values and keep only distances with a space. Ignores instances with only street names as this is not an address.
print(df.shape)
df =df[df['location2'].isna()==False]
df = df[df['location2'].apply(lambda x: x.isdigit()==False)]
df = df[df['location2'].apply(lambda x: x.find(" ")==-1)==False]
print(df.shape)

(2213939, 11)
(2213689, 11)


In [7]:
#Format date and delete invalid dates. Delete Saturdays and Sundays.
df['year'] = df['date_of_infraction'].apply(lambda x: int(str(x)[0:4]))
df['month'] = df['date_of_infraction'].apply(lambda x: int(str(x)[4:6]))
df['day'] = df['date_of_infraction'].apply(lambda x: int(str(x)[6:8]))
df['formatted_date'] = pd.to_datetime(df[['year','month','day']])
df['day_of_week'] = df['formatted_date'].dt.dayofweek
df['day_name'] = df['formatted_date'].dt.day_name()

df = df[df['day_name']!='Saturday']
df = df[df['day_name']!='Sunday']

In [8]:
df.shape

(1716858, 17)

In [9]:
addresses = df['location2']
addresses.head()

0          2110 KEELE ST
1    125 QUEENS WHARF RD
2          HAZELWOOD AVE
3         10 CARABOB CRT
4    1801 EGLINTON AVE W
Name: location2, dtype: object

In [10]:
streets_of_interest = pd.read_csv("C:\\Users\\stefy\\OneDrive - University of Waterloo\\Work Term Report 3 - Parking\\Datasets\\Toronto\\Toronto Centerline\\CENTRELINE_WGS84.csv")
streets_of_interest = streets_of_interest[streets_of_interest['LINEAR_NAME_FULL']!= 'Planning Boundary']
streets_of_interest = streets_of_interest[streets_of_interest['LINEAR_NAME_FULL']!= 'C N R']
streets_of_interest = streets_of_interest[streets_of_interest['LINEAR_NAME_FULL']!= 'C P R']
streets_of_interest = streets_of_interest[streets_of_interest['LINEAR_NAME_FULL']!= 'Toronto Shoreline']

streets_of_interest.head(3)

,CENTRELINE_ID,LINEAR_NAME_ID,LINEAR_NAME_FULL,LINEAR_NAME_FULL_LEGAL,ADDRESS_L,ADDRESS_R,PARITY_L,PARITY_R,LO_NUM_L,HI_NUM_L,...,TO_INTERSECTION_ID,ONEWAY_DIR_CODE,ONEWAY_DIR_CODE_DESC,FEATURE_CODE,FEATURE_CODE_DESC,JURISDICTION,CENTRELINE_STATUS,OBJECTID,MI_PRINX,Shape__Length
0,14025592,3147,Charlotte St,Charlotte Street,NaN,NaN,N,N,NaN,NaN,...,14025595,0,Not One-Way,201500,Local,CITY OF TORONTO,NaN,18733,18733,6.292203
1,1146472,4733,Yonge St,Yonge Street,NaN,NaN,N,N,NaN,NaN,...,13467668,0,Not One-Way,201200,Major Arterial,CITY OF TORONTO,NaN,18852,18852,6.219553
3,20054575,436,Bathurst St,Bathurst Street,NaN,NaN,N,N,NaN,NaN,...,20054577,0,Not One-Way,201200,Major Arterial,CITY OF TORONTO,NaN,50548,50548,4.816115


In [6]:
streets_of_interest_list = streets_of_interest['LINEAR_NAME_FULL'].unique()
streets_of_interest_list

array(['Charlotte St', 'Yonge St', 'Bathurst St', 'Church St',
       'Lower Simcoe St', 'White Court Pl', 'Orphanage Mews',
       'Canoe Landing Park Trl', 'Sullivan St', 'Nelson St',
       'Bremner Blvd', 'Grange Ave', 'Spadina Ave', 'Jarvis St',
       'Martin Goodman Trl', 'Ln E Spadina N Phoebe',
       'Ln S Dundas W Larch', 'Alex Wilson Community Garden Trl',
       'Huron St', 'Lakeshore Bv W Uturn Ramp', 'F G Gardiner Xy W',
       'Dan Leckie Way', 'Sheard St', 'Joseph Sheard Parkette Trl',
       'Toronto Music Garden Trl', "St. Andrew's Playground Trl",
       'St. James Park Trl', 'Little Norway Park Trl', 'Eireann Quay',
       'St. Patricks Square Trl', 'Grange Park Trl', 'Carr Street Crt',
       'Robertson Cres', 'Lake Shore Blvd W', 'Clarence Sq',
       'Alexandra Park Trl', 'School Lane', 'Phoebe St', 'Ryerson Ave',
       'Mc Caul St', 'York St', 'Arena Gardens Trl', 'King St E',
       'John St', 'Glasgow St', 'Ln S Cecil W Huron', 'Lower Jarvis St',
       'The

In [12]:
delimited_rows = list()
for street in streets_of_interest_list:
    street = street.lower()
    for index, row in df.iterrows():
        address = row['location2'].lower()
        try:
            if address.find(street)>-1:
                delimited_rows.append(row)
                print(address, street)
        except:
            print(address, street)
        
delimited_df = pd.DataFrame(delimited_rows, columns=df.columns)

50 mutual st mutual st
150 mutual st mutual st
160 mutual st mutual st
123 mutual st mutual st
123 mutual st mutual st
123 mutual st mutual st
123 mutual st mutual st
mutual st mutual st
285 mutual st mutual st
301 mutual st mutual st
231 mutual st mutual st
221 mutual st mutual st
209 mutual st mutual st
219 mutual st mutual st
271 mutual st mutual st
261 mutual st mutual st
25 mutual st mutual st
285 mutual st mutual st
160 mutual st mutual st
140 mutual st mutual st
108 mutual st mutual st
25 mutual st mutual st
88 mutual st mutual st
78 mutual st mutual st
160 mutual st mutual st
160 mutual st mutual st
145 mutual st mutual st
199 mutual st mutual st
309 mutual st mutual st
215 mutual st mutual st
238 mutual st mutual st
238 mutual st mutual st
123 mutual st mutual st
150 mutual st mutual st
150 mutual st mutual st
160 mutual st mutual st
25 mutual st mutual st
285 mutual st mutual st
25 mutual st mutual st
211 mutual st mutual st
mutual st mutual st
mutual st mutual st
mutual st m

MemoryError: Unable to allocate 105. MiB for an array with shape (8, 1716858) and data type object

In [33]:
"Toronto".find("te")

-1